# 01 - BigQuery - Table Data Source

Taken and adapted from [Vertex Ai Mlops](https://github.com/statmike/vertex-ai-mlops), [Vertex AI Pipelines](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_classification_model_evaluation.ipynb).


---
## Source Data

[Heart Disease Health Indicators Dataset](https://www.kaggle.com/datasets/alexteboul/heart-disease-health-indicators-dataset?resource=download) from Kaggle.

---
## Setup

inputs:

In [45]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'heart-disease-classification-1'

In [46]:
REGION = 'europe-west1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'heart'
BQ_TABLE = 'heart'

# # Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

packages:

In [47]:
from google.cloud import bigquery
from google.cloud import storage

clients:

In [48]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

parameters:

In [49]:
BUCKET = PROJECT_ID

## Store the Source Data in GCS Storage Bucket from Kaggle

In [86]:
!#pip install opendatasets

In [87]:
source_folder = "heart-disease-health-indicators-dataset"
filename = "heart_disease_health_indicators_BRFSS2015"
LOCAL_LOCATION = f"{source_folder}/{filename}.csv"
import os 
import pandas as pd

In [ ]:
import opendatasets as od
od.download(
    "https://www.kaggle.com/datasets/alexteboul/heart-disease-health-indicators-dataset")


In [89]:
# create a new datset
bq.create_dataset(BQ_DATASET)

Dataset(DatasetReference('heart-disease-classification-1', 'heart'))

In [90]:
# create a new table in that dataset ()
bq.create_table(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")

Table(TableReference(DatasetReference('heart-disease-classification-1', 'heart'), 'heart'))

In [91]:
FILE_LOCATION = f"{source_folder}/{filename}.csv"

# dataset_ref = bq.dataset(BQ_DATASET)
# table_ref = dataset_ref.table(table_id)
destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )

# load the csv into bigquery
with open(FILE_LOCATION, "rb") as source_file:
    #job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    job = bq.load_table_from_file(source_file, destination, job_config = job_config)
    
job.result()  # Waits for table load to complete.
print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

Finished creating table: heart-disease-classification-1.heart.heart


In [92]:
# make booleans from floats for classification
df = pd.read_csv(f"{source_folder}/{filename}.csv")
df['HeartDiseaseorAttack'] = df['HeartDiseaseorAttack'].astype('bool')
df.to_csv(f"{source_folder}/{filename}.csv", index=False)

In [95]:
print(f'Review the files in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')

Review the files in the console here:
https://console.cloud.google.com/storage/browser/heart-disease-classification-1/01;tab=objects&project=heart-disease-classification-1


### Retrieve and Review a Sample From The Table:
> **Note:** The `LIMIT 5` statement does limit the number of rows returned by BigQuery to 5 but BigQuery still does a full table scan.  If you have a table larger than 1GB and want to limit the rows scanned for a quick review like then then replacing `LIMIT 5` with `TABLESAMPLE SYSTEM (1 PERCENT)` would be more efficient.  For tables under 1GB it will still return the full table.  More on [Table Sampling](https://cloud.google.com/bigquery/docs/table-sampling)

In [102]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
LIMIT 5
"""
bq.query(query = query).to_dataframe()

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,False,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,15.0,0.0,0.0,0.0,2.0,6.0,7.0
1,False,0.0,0.0,1.0,22.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,5.0
2,False,0.0,1.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,4.0,2.0,0.0,0.0,2.0,6.0,8.0
3,False,0.0,1.0,1.0,32.0,1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,2.0,10.0,0.0,0.0,0.0,2.0,5.0,2.0
4,False,0.0,0.0,0.0,32.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,2.0,30.0,0.0,0.0,0.0,2.0,5.0,1.0


### Check out this table in BigQuery Console:
- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

In [103]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=heart-disease-classification-1


---
## Review Data in BigQuery
Additional SQL queries could be used to review the data.  This section shows moving the table to a Pandas dataframe for local review in Python:

> **Note:** <p>This query only selects one column.  This means BigQuery scans less data as it does not process the other columns.  </p>

In [104]:
query = f"""
SELECT HeartDiseaseorAttack
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [105]:
df['HeartDiseaseorAttack'].value_counts()

False    229787
True      23893
Name: HeartDiseaseorAttack, dtype: int64

In [106]:
df['HeartDiseaseorAttack'].value_counts(normalize=True)

False    0.905814
True     0.094186
Name: HeartDiseaseorAttack, dtype: float64

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [107]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [108]:
(job.ended-job.started).total_seconds()

4.312

In [109]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

42.87192 MB


Review the test/train split:

In [110]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,VALIDATE,25583,10.084752
1,TRAIN,203085,80.055582
2,TEST,25012,9.859666


Retrieve a subset of the data to a Pandas dataframe:

In [111]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [112]:
data.head()

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,transaction_id,splits
0,False,0.0,0.0,1.0,33.0,0.0,0.0,2.0,1.0,1.0,...,3.0,0.0,0.0,0.0,0.0,4.0,2.0,4.0,c197c571-59c4-40f8-b9f0-8619852e950b,TRAIN
1,False,0.0,0.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,...,3.0,5.0,5.0,0.0,0.0,4.0,2.0,4.0,b22bf429-aa71-47b5-9424-4146281538cb,TRAIN
2,False,0.0,0.0,1.0,32.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,4.0,2.0,2.0,b64c116e-ccaa-4f08-b400-c9c3b125d27c,TRAIN
3,False,0.0,1.0,1.0,33.0,0.0,0.0,2.0,0.0,0.0,...,3.0,7.0,0.0,0.0,0.0,5.0,2.0,1.0,69e4f8ba-0288-45e6-b9cf-396d9de68916,TRAIN
4,False,0.0,1.0,1.0,29.0,1.0,0.0,1.0,0.0,0.0,...,4.0,0.0,25.0,0.0,0.0,5.0,2.0,1.0,2f452d92-f0cc-4cb7-9019-a2ab1c24b5c5,TRAIN
